<a href="https://colab.research.google.com/github/SKM1978/Deep_Learning_Tutorial/blob/master/CNN_using_keras_tuner_Fashion_MNIST_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 97 kB 2.7 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.6.0


## Uploading the Fashion MNIST dataset

In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


## Scaling down the images for easier computation

In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
# shape of the data

train_images.shape, train_images[0].shape

((60000, 28, 28), (28, 28))

In [8]:
# Reshaping the images

train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

## Building the CNN network

In [9]:
def build_model(hp):
  model=keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values=(3,5)),
        activation='relu',
        input_shape=(28,28,1)),

    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values=(3,5)),
        activation='relu'),

    keras.layers.Flatten(),

    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'),

    keras.layers.Dense(10, activation='softmax')])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy', 
                metrics=['accuracy'])

  return model

## Using Random Search to find the best HyperParameters

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [11]:
tuner_search=RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name='fashion mnist')

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 01m 22s]
val_accuracy: 0.9100000262260437

Best val_accuracy So Far: 0.9116666913032532
Total elapsed time: 00h 06m 44s
INFO:tensorflow:Oracle triggered exit


In [13]:
# selecting the best parameters that gives highest accuracy for the model

model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
# which are the best parameters selected for the model

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        115248    
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2230368   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,347,546
Trainable params: 2,347,546
Non-trainable params: 0
_________________________________________________________________


## Training the model with those best parameters

In [16]:
# Training this model

model.fit(train_images, train_labels, epochs=50, validation_split=0.1, initial_epoch=3)

Epoch 4/50
1688/1688 [==============================] - 17s 10ms/step - loss: 0.1000 - accuracy: 0.9625 - val_loss: 0.2817 - val_accuracy: 0.9122
Epoch 5/50
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0673 - accuracy: 0.9749 - val_loss: 0.3238 - val_accuracy: 0.9105
Epoch 6/50
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0495 - accuracy: 0.9818 - val_loss: 0.4098 - val_accuracy: 0.9123
Epoch 7/50
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0359 - accuracy: 0.9869 - val_loss: 0.4067 - val_accuracy: 0.9110
Epoch 8/50
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0308 - accuracy: 0.9889 - val_loss: 0.4928 - val_accuracy: 0.9108
Epoch 9/50
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0252 - accuracy: 0.9912 - val_loss: 0.4785 - val_accuracy: 0.9097
Epoch 10/50
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0223 - accuracy: 0.9918 - val_loss: 0.5757 

### The model with the best HyperParameters give us 99% accuracy and 90% validation accuracy. This was possible with the library keras-tuner.